# Imports and environment setup

In [102]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
#from PIL import ImageGrab
import cv2
import time
from mss import mss

#mouse control and window dimensions
SCREEN_WIDTH = 1280
SCREEN_HEIGHT = 1024
import win32api, win32con
#Player Control
click_cooldown = 0.5

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from utils import label_map_util
from utils import visualization_utils as vis_util

# Load Model

In [103]:
#Select model in the same folder as this script
MODEL_NAME = 'LeagueAI_v3'
MODEL_FILE = MODEL_NAME + '.tar.gz'
#DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('training', 'LeagueAI_v2.pbtxt')

NUM_CLASSES = 3

## Load the frozen model into memory

In [105]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map

In [106]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [107]:
## Other

In [109]:
## Champion helper classes
# click a postion in the screen using x,y coordinates
def click(x,y):
    win32api.SetCursorPos((x,y))
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN,x,y,0,0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP,x,y,0,0)
# find the player character position marked as x,y (converting the boxes to a single point position)
def find_player_xy(boxes):
    playerwidth =  (boxes[3] - boxes[1])
    playerheight = boxes[2] - boxes[0]
    playerpos_y = (boxes[0] + playerheight/1.3)
    playerpos_x = (boxes[1] + playerwidth/2)
    return([playerpos_y, playerpos_x, playerheight, playerwidth])
# click a certain state given by x and y coordinate in the state grid
def click_state(state_x, state_y, state_width, state_height, player_origin):
    x_click = (player_origin[0]/800)*SCREEN_WIDTH + state_x*(state_width/800)*SCREEN_WIDTH
    y_click = (player_origin[1]/450)*SCREEN_HEIGHT + (-1*state_y*(state_height/450)*SCREEN_HEIGHT)
    print(x_click)
    click(int(x_click), int(y_click))

In [ ]:
## Intelligence
# TODO

## Main Routine

In [108]:
sct = mss()
# determine the size of the area to observe (the game)
mon = {'top': 0, 'left': 0, 'width': SCREEN_WIDTH, 'height': SCREEN_HEIGHT}
# timer to see how fast the loop is running
last_time = time.time()
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
      print('Loop took: {} seconds'.format(time.time()-last_time))
      last_time = time.time()
      #screen = cv2.resize(grab_screen(region=(0,40,1024,768)), (800,450))
      #screen = np.array(ImageGrab.grab(bbox=(0,40,1024,768)))
      # get screen recording and resize it to 800x450 pixels for output
      sct.get_pixels(mon)
      screen = Image.frombytes('RGB', (sct.width, sct.height), sct.image)
      screen = np.array(screen)
      screen = cv2.resize(screen, (800,450))
      image_np = screen
      # ===================tensorflow code===============
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})                  
      
      # ====================================================  
      # Visualization of the results of a detection
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          min_score_thresh=0.25,
          line_thickness=4)

      #===================Player Character=======================
      gameover = False
      vaynefound = False
      for i,b in enumerate(boxes[0]):
          if scores[0][i] >= 0.25:
              if classes[0][i] == 1 and vaynefound == False:
                  vaynefound = True
                  player_position = find_player_xy(boxes[0][i])
                  #click(int(player_position[1]*SCREEN_WIDTH), int(player_position[0]*SCREEN_HEIGHT))
                  #draw a circle where the character is
                  player_origin = [int(player_position[1]*800), int(player_position[0]*450)]
                  cv2.circle(image_np, (player_origin[0], player_origin[1]), 2, (0, 0, 255), 2)
                  #calculate the origin of the center rectangle under the character
                  origin = [int(player_origin[0]+9)-int(player_position[2]*800/3), player_origin[1]-15,int(player_origin[0])+20, int(player_origin[1])+18]        
                  cv2.rectangle(image_np,(origin[0], origin[1]),(origin[2], origin[3]), (0, 255, 0), 1)
                  #create draw states
                  w = origin[2] - origin[0]
                  h = origin[3] - origin[1]
                  grid_x = int(SCREEN_WIDTH/w)
                  grid_y = int(SCREEN_HEIGHT/h)
                  for x in range(grid_x):
                        for y in range(grid_y):
                            #cv2.rectangle(image_np,(origin[0]+w*x, origin[1]+h*y),(origin[2]+w*x, origin[3]+h*y), (0, 255, 0), 1)
                            cv2.rectangle(image_np,(origin[0]+w*x-int(grid_x/2)*w, origin[1]+h*y-int(grid_y/2)*h),(origin[2]+w*x-int(grid_x/2)*w, origin[3]+h*y-int(grid_y/2)*h), (0, 255, 0), 1)
                  click_state(1,1,w,h, player_origin)
      # ==================OPEN CV visualization=================
      cv2.imshow('window',image_np)
      if (cv2.waitKey(25) & 0xFF == ord('q')):
          cv2.destroyAllWindows()
          break
      if gameover == True:
          print('gameover')
          cv2.destroyAllWindows()
          break;


Loop took: 0.033620595932006836 seconds
Loop took: 2.494089126586914 seconds
Loop took: 0.796116828918457 seconds
Loop took: 0.781578540802002 seconds
Loop took: 0.7971169948577881 seconds
Loop took: 0.7605235576629639 seconds
Loop took: 0.7635347843170166 seconds
Loop took: 0.7615220546722412 seconds
Loop took: 0.7770676612854004 seconds
Loop took: 0.7615294456481934 seconds
Loop took: 0.7926020622253418 seconds
Loop took: 0.8080525398254395 seconds
Loop took: 0.7770669460296631 seconds
Loop took: 0.7770671844482422 seconds
Loop took: 0.7615265846252441 seconds
Loop took: 0.7459845542907715 seconds
Loop took: 0.7770709991455078 seconds
Loop took: 0.7459805011749268 seconds
Loop took: 0.808110237121582 seconds
Loop took: 0.807647705078125 seconds
Loop took: 0.7570140361785889 seconds
Loop took: 0.7770686149597168 seconds
Loop took: 0.7770657539367676 seconds
Loop took: 0.792609453201294 seconds
Loop took: 0.7615251541137695 seconds
Loop took: 0.7615275382995605 seconds
Loop took: 0.761